In [0]:
# dbutils.library.installPyPI("keras")
# dbutils.library.installPyPI("tensorflow")
# dbutils.library.installPyPI("image")

# dbutils.library.restartPython()

NameError: name 'dbutils' is not defined

In [0]:
# Standard Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Pipelines for training and evaluating model
from sklearn.pipeline import Pipeline

# Preprocessing/Feature Extraction
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Models 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

# Scoring
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

import requests
from PIL import Image
from io import BytesIO
from keras.wrappers.scikit_learn import KerasClassifier
from keras.backend import clear_session

Using TensorFlow backend.


In [0]:
# Converts JSON dictionary of Video Category Ids for 
# a given country into a DataFrame
def json_to_df(path):
    f = lambda x: x.loc['items']['snippet']['title']
    g = lambda x: x.loc['items']['snippet']['assignable']
    h = lambda x: x.loc['items']['id']
    
    df = pd.read_json(path)
    df['Category'] = df.apply(f, axis = 1)
    df['Assignable'] = df.apply(g, axis = 1)
    df['Id'] = df.apply(h, axis = 1)
    df['Id'] = df['Id'].astype(int)
    
    cols = ['Id', 'Category', 'Assignable']
    
    df = df[cols]
    
    return df[df['Assignable']]

In [0]:
countries = ['CA', 'GB', 'US']
data = pd.DataFrame({})

# Concatenating Data
for country in countries:
  
    country_youtube_video_data_path = 'youtube-new/{}videos.csv'.format(country)
    country_category_id_path ='youtube-new/{}_category_id.json'.format(country)
    
    country_category_id_data = json_to_df(country_category_id_path)
    country_youtube_video_data = pd.read_csv(country_youtube_video_data_path)
    
    country_youtube_video_data['category_id'] = country_youtube_video_data['category_id'].astype(int)
    
    video_data_w_category = pd.merge(country_youtube_video_data, 
                                     country_category_id_data,
                                     left_on = 'category_id',
                                     right_on = 'Id')
    
    data = pd.concat([data, video_data_w_category])
    
data['tags'] = (data['tags']
                .str
                .replace('|', ' ')
                .str
                .replace('"', '')
               )
data.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Id,Category,Assignable
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,Eminem Walk On Water Aftermath/Shady/Interscop...,17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,10,Music,True
1,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,edsheeran ed sheeran acoustic live cover offic...,33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,10,Music,True
2,H1KBHFXm2Bg,17.14.11,21 Savage - Bank Account (Official Music Video),21 Savage,10,2017-11-10T19:00:02.000Z,21 savage bank account 21 savage bank account ...,5068229,263596,8585,28976,https://i.ytimg.com/vi/H1KBHFXm2Bg/default.jpg,False,False,False,Watch the official music video of Bank Account...,10,Music,True
3,7MxiQ4v0EnE,17.14.11,Daang ( Full Video ) | Mankirt Aulakh | Sukh S...,Speed Records,10,2017-11-11T16:41:15.000Z,punjabi songs punjabi bhangra punjabi music pu...,5718766,127477,7134,8063,https://i.ytimg.com/vi/7MxiQ4v0EnE/default.jpg,False,False,False,Song - Daang\nSinger - Mankirt Aulakh\nFaceboo...,10,Music,True
4,j67FgNEvyh8,17.14.11,Telefoon || Babbu Maan || Promo || Full Song 1...,Hey Yolo,10,2017-11-12T04:24:34.000Z,Telefoon Babbu Maan Mehfil Mitran Di new punja...,178447,4339,96,876,https://i.ytimg.com/vi/j67FgNEvyh8/default.jpg,False,False,False,"Full Song Out Now 15th November\n\nHey Yolo, S...",10,Music,True


In [0]:
data.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Id,Category,Assignable
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,Eminem Walk On Water Aftermath/Shady/Interscop...,17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,10,Music,True
1,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,edsheeran ed sheeran acoustic live cover offic...,33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,10,Music,True
2,H1KBHFXm2Bg,17.14.11,21 Savage - Bank Account (Official Music Video),21 Savage,10,2017-11-10T19:00:02.000Z,21 savage bank account 21 savage bank account ...,5068229,263596,8585,28976,https://i.ytimg.com/vi/H1KBHFXm2Bg/default.jpg,False,False,False,Watch the official music video of Bank Account...,10,Music,True
3,7MxiQ4v0EnE,17.14.11,Daang ( Full Video ) | Mankirt Aulakh | Sukh S...,Speed Records,10,2017-11-11T16:41:15.000Z,punjabi songs punjabi bhangra punjabi music pu...,5718766,127477,7134,8063,https://i.ytimg.com/vi/7MxiQ4v0EnE/default.jpg,False,False,False,Song - Daang\nSinger - Mankirt Aulakh\nFaceboo...,10,Music,True
4,j67FgNEvyh8,17.14.11,Telefoon || Babbu Maan || Promo || Full Song 1...,Hey Yolo,10,2017-11-12T04:24:34.000Z,Telefoon Babbu Maan Mehfil Mitran Di new punja...,178447,4339,96,876,https://i.ytimg.com/vi/j67FgNEvyh8/default.jpg,False,False,False,"Full Song Out Now 15th November\n\nHey Yolo, S...",10,Music,True


In [0]:
le = LabelEncoder()
X, y = data.drop(['Category'], axis = 1), data['Category']
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 420)

print('Dataset Length: {}'.format(len(X)),'Training Data Length: {}'.format(len(X_train)),'Number of Response Categories: {}'.format(len(np.unique(y_train))))

Dataset Length: 120375 Training Data Length: 90281 Number of Response Categories: 15


In [0]:
def get_thumbnail(url): 
    r = requests.get(url).content
    img_data = Image.open(BytesIO(r))
    img_data = img_data.resize((90, 90))
    
    return np.array(img_data).reshape(1, 90, 90, 3)

# Download the Video thumbnails to use to predict video's category
img_data = X_train['thumbnail_link'].map(get_thumbnail)

img_data.head()

In [0]:
img_data = np.concatenate([img_data.iloc[i] for i in range(len(img_data))])
img_data.shape

In [0]:
# Convert class labels into class vectors
y_train_img_model = np.array([y_train[i]
                              for i in range(len(y_train))
                              if i in good_img_indices])
                              
y_train_img_model = keras.utils.to_categorical(y_train_img_model, num_classes = 15)
y_train_img_model

In [0]:
# Dictionary to store different models' performances
model_dict = {'Model':[], 'Training Acc.':[], 'Validation Acc.':[]}

# Return training error and validation error for model
def get_train_val_error(model):
    scorer = make_scorer(accuracy_score)
    
    model.fit(X_train, y_train)

    preds = model.predict(X_train)
    training_acc = accuracy_score(y_train, preds)
    val_acc = np.mean(cross_val_score(model, X_train, y_train, scoring = scorer))

    return training_acc, val_acc

In [0]:
cols = ['title', 'tags', 'description']

# Data Pipeline for text-based model
model_text = Pipeline([
    ('Columns', ColumnTransformer([
        ('keep', 'passthrough', cols)
    ])),
    ('Imputation', SimpleImputer(strategy = 'constant', fill_value = '')),
    ('Bag of Words', ColumnTransformer([
        (cols[0], CountVectorizer(stop_words = 'english', max_features = 10000), 0),
        (cols[1], CountVectorizer(stop_words = 'english', max_features = 10000), 1),
        (cols[2], CountVectorizer(stop_words = 'english', max_features = 10000), 2)
    ])),
    ('TF-IDF', TfidfTransformer()),
    ('Model Text', LogisticRegression(multi_class = 'multinomial', solver = 'sag'))
])

model_text

In [0]:
# Get training and validation error of text-based model
model_text_train_err, model_text_val_err = get_train_val_error(model_text)

print('Training Accuracy: {}'.format(model_text_train_err))
print('Validation Accuracy: {}'.format(model_text_val_err))

In [0]:
# Add text-based model's performance to the model dictionary
model_dict['Model'].append('Text Model (Baseline)')
model_dict['Training Acc.'].append(model_text_train_err)
model_dict['Validation Acc.'].append(model_text_val_err)

In [0]:
# Clear previous model weights and returns a blank baseline CNN model
# consisting of 10 feature maps, a 3x3 local receptive field, and the ReLU
# activaton function
def baseline_image_model():
    clear_session()
    
    model = Sequential()
    model.add(Conv2D(10, (3, 3), activation='relu', input_shape = (50, 50, 3)))
    model.add(Flatten())
    model.add(Dense(15, activation='softmax'))

    sgd = SGD(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics = ['accuracy'])
    
    return model

image_model = baseline_image_model()
image_model

Out[23]: <keras.engine.sequential.Sequential at 0x7f2e49d9af28>

In [0]:
# Return validation accuracies for image-based model
def cross_val_score_custom(X_train_img, y_train_img):
    from sklearn.model_selection import KFold
    
    kfold = KFold(n_splits = 3)
    val_accs = []
    
    for train_index, val_index in kfold.split(X_train_img):
        new_model = baseline_image_model()
        
        new_model.fit(X_train_img[train_index],
                      y_train_img[train_index],
                      epochs = 5,
                      batch_size = 128)
        
        val_error, val_acc = new_model.evaluate(X_train_img[val_index], y_train_img[val_index])
        val_accs.append(val_acc)
        
    return val_accs

# Return training and validation accuracy of image-based model
def get_train_val_error_image_model(image_model):
    image_model.fit(img_data / 255, y_train_img_model, epochs = 5)
    
    train_err, train_acc = image_model.evaluate(img_data / 255, y_train_img_model)
    val_acc = np.mean(cross_val_score_custom(img_data / 255, y_train_img_model))
    
    return train_acc, val_acc

# Get training and validation error of image-based model
train_acc_image_model, val_acc_image_model = get_train_val_error_image_model(image_model)

Epoch 1/5

 32/90281 [..............................] - ETA: 4:00 - loss: 2.8722 - accuracy: 0.0312
 96/90281 [..............................] - ETA: 2:11 - loss: 2.7460 - accuracy: 0.1250
 160/90281 [..............................] - ETA: 2:00 - loss: 2.6740 - accuracy: 0.1375
 224/90281 [..............................] - ETA: 1:48 - loss: 2.6090 - accuracy: 0.1830
 288/90281 [..............................] - ETA: 1:43 - loss: 2.5848 - accuracy: 0.1875
 352/90281 [..............................] - ETA: 1:38 - loss: 2.5654 - accuracy: 0.1960
 416/90281 [..............................] - ETA: 1:35 - loss: 2.5594 - accuracy: 0.1947
 480/90281 [..............................] - ETA: 1:32 - loss: 2.5544 - accuracy: 0.1896
 544/90281 [..............................] - ETA: 1:31 - loss: 2.5294 - accuracy: 0.1967
 608/90281 [..............................] - ETA: 1:29 - loss: 2.5110 - accuracy: 0.2023
 672/90281 [..............................] - ETA: 1:28 - loss: 2.4914 - accuracy: 0.2083
 736/90281 [..............................] - ETA: 1:27 - loss: 2.4750 - accuracy: 0.2188
 800/90281 [..............................] - ETA: 1:26 - loss: 2.4689 - accuracy: 0.2138
 864/90281 [..............................] - ETA: 1:26 - loss: 2.4581 - accuracy: 0.2176
 928/90281 [..............................] - ETA: 1:25 - loss: 2.4497 - accuracy: 0.2209
 992/90281 [..............................] - ETA: 1:25 - loss: 2.4433 - accuracy: 0.2248
 1056/90281 [..............................] - ETA: 1:24 - loss: 2.4320 - accuracy: 0.2292
 1120/90281 [..............................] - ETA: 1:23 - loss: 2.4237 - accuracy: 0.2348
 1184/90281 [..............................] - ETA: 1:23 - loss: 2.4195 - accuracy: 0.2390
 1248/90281 [..............................] - ETA: 1:23 - loss: 2.4154 - accuracy: 0.2396
 1312/90281 [..............................] - ETA: 1:22 - loss: 2.4050 - accuracy: 0.2454
 1376/90281 [..............................] - ETA: 1:22 - loss: 2.3946 - accuracy: 0.2500
 1440/90281 [..............................] - ETA: 1:22 - loss: 2.3871 - accuracy: 0.2556
 1504/90281 [..............................] - ETA: 1:22 - loss: 2.3832 - accuracy: 0.2566
 1568/90281 [..............................] - ETA: 1:21 - loss: 2.3820 - accuracy: 0.2551
 1632/90281 [..............................] - ETA: 1:21 - loss: 2.3827 - accuracy: 0.2543
 1696/90281 [..............................] - ETA: 1:21 - loss: 2.3818 - accuracy: 0.2559
 1760/90281 [..............................] - ETA: 1:21 - loss: 2.3752 - accuracy: 0.2580
 1824/90281 [..............................] - ETA: 1:20 - loss: 2.3682 - accuracy: 0.2593
 1888/90281 [..............................] - ETA: 1:20 - loss: 2.3616 - accuracy: 0.2606
 1952/90281 [..............................] - ETA: 1:20 - loss: 2.3584 - accuracy: 0.2654
 2016/90281 [..............................] - ETA: 1:20 - loss: 2.3571 - accuracy: 0.2654
 2080/90281 [..............................] - ETA: 1:20 - loss: 2.3553 - accuracy: 0.2668
 2144/90281 [..............................] - ETA: 1:20 - loss: 2.3519 - accuracy: 0.2673
 2208/90281 [..............................] - ETA: 1:20 - loss: 2.3576 - accuracy: 0.2654
 2272/90281 [..............................] - ETA: 1:20 - loss: 2.3582 - accuracy: 0.2667
 2336/90281 [..............................] - ETA: 1:19 - loss: 2.3565 - accuracy: 0.2684
 2400/90281 [..............................] - ETA: 1:19 - loss: 2.3523 - accuracy: 0.2713
 2464/90281 [..............................] - ETA: 1:19 - loss: 2.3479 - accuracy: 0.2727
 2528/90281 [..............................] - ETA: 1:19 - loss: 2.3507 - accuracy: 0.2722
 2592/90281 [..............................] - ETA: 1:19 - loss: 2.3466 - accuracy: 0.2735
 2656/90281 [..............................] - ETA: 1:19 - loss: 2.3419 - accuracy: 0.2752
 2720/90281 [..............................] - ETA: 1:19 - loss: 2.3434 - accuracy: 0.2757
 2784/90281 [..............................] - ETA: 1:18 - loss: 2.3442 - accuracy: 0.2762
 28

In [0]:
print('Training accuracy for baseline image-based model: {}'.format(train_acc_image_model))
print('Validation accuracy for baseline image-based model: {}'.format(val_acc_image_model))

Training accuracy for baseline image-based model: 0.4247737526893616
Validation accuracy for baseline image-based model: 0.34779177109400433

In [0]:
# Add image-based model's performance to the model dictionary

model_dict['Model'].append('Img Model (Baseline)')
model_dict['Training Acc.'].append(train_acc_image_model)
model_dict['Validation Acc.'].append(val_acc_image_model)

In [0]:
model_performance = pd.DataFrame(model_dict, index = model_dict['Model'])
model_performance = model_performance.iloc[[0, 2]]
model_performance

,Model,Training Acc.,Validation Acc.
Text Model (Baseline),Text Model (Baseline),0.965065,0.934726
Img Model (Baseline),Img Model (Baseline),0.424774,0.347792


In [0]:
# Setting the positions and width for the bars
pos = list(range(len(model_performance))) 
width = 0.25 
    
# Plotting the bars
fig, ax = plt.subplots(figsize = (12,6))

plt.bar(pos, 
        #using df['pre_score'] data,
        model_performance['Training Acc.'], 
        # of width
        width, 
        # with alpha 0.5
        alpha=0.5, 
        # with color
        color='#EE3224', 
        align = 'center',
        # with label the first value in first_name
        label=model_performance['Model'][0]) 

plt.bar([p + width for p in pos], 
        #using df['mid_score'] data,
        model_performance['Validation Acc.'],
        # of width
        width, 
        # with alpha 0.5
        alpha=0.5, 
        # with color
        color='#F78F1E',
        align = 'center',
        # with label the second value in first_name
        label=model_performance['Model'][1]) 

# Set the y axis label
ax.set_ylabel('Accuracy')

# Set the chart's title
ax.set_title('Training and Validation Accuracies for Img and Text-based Models')

# Set the position of the x ticks
ax.set_xticks([p + 0.5 * width for p in pos])

# Set the labels for the x ticks
ax.set_xticklabels(model_performance['Model'], ha = 'center')


# Adding the legend and showing the plot
plt.legend(['Training Acc.', 'Validation Acc.'], loc='upper right')

display()